The purpose of this notebook is to provide a unified interface to our code for doing data generation.

We plan to do experiment with the following things:

**Circuit**

We will use circuits 1, 6, 9, and 14, as they cover a variety of circuit expressibilities and entangling capabilities.

| | High Ent | Low Ent |
| --- | --- | --- |
| High Exp|6 | 14 |
|Low Exp| 9 | 1 |

**Data sets**

We will use 4 data sets:

* Data set 0 (2 blobs, separable)
* Data set 1 (2 blobs, inseparable)
* Data set 2a (4 blobs, separable)
* Data set 3c (4 blobs) <-- Saesun's choice

**Learning rates**

We will investigate several learning rates.

----------------

The code below will help us set up our experiments.

In [1]:
# The code below is a hack in case Travis' kernel fails.
#import os
#os.environ['KMP_DUPLICATE_LIB_OK'] ='True'

# Pull in the helper files.
%run experiments

C:\Users\kim8104\Anaconda3\envs\my-torch\lib\site-packages\qiskit\__init__.py:67: DeprecationWarning: Using Qiskit with Python 3.6 is deprecated as of the 0.17.0 release. Support for running Qiskit with Python 3.6 will be removed in a future release.
  "future release.", DeprecationWarning)


In [2]:
# Set up the experiment
circuitID = 14
epochs = 20

In [ ]:
import numpy
import sys
numpy.set_printoptions(threshold=sys.maxsize)
lr_list=[round(((i+1)/10)**2,2) for i in range(20)]
ds_list=['0','1a','2a','3c']

for dsID in ds_list:
# Run the experiment
    print('--------dataset',dsID,'is initialized------')
    lr_acc=[]
    for lr in lr_list:
        # Load in the data
        data = load_data(dsID)

        # Generate the splittings
        train_X, train_y, validate_X, validate_y, test_X, test_y = generate_train_validate_test_data(data)

        # Make the feature map
        feature_map= make_embedding_circuit()

        # Make the classifier
        ansatz = make_classifer_circuit(circuitID)

        # Do the training
        model = train_model(feature_map, ansatz, epochs, lr, train_X, train_y)

        # Check the validation accuracy.
        val_accuracy = check_accuracy(model, validate_X, validate_y)

        lr_acc.append([lr,val_accuracy])
    np.savetxt(r"Learning_Rate_Data\circuit{0}_data{1}.txt".format(circuitID,dsID),lr_acc,fmt='%.2f')

--------dataset 0 is initialized------
__Learning Rate ( 0.01 ) is intialized


C:\Users\kim8104\Anaconda3\envs\my-torch\lib\site-packages\torch\nn\modules\loss.py:528: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 2])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


__Learning Rate ( 0.01 ) is done
__Learning Rate ( 0.04 ) is intialized
__Learning Rate ( 0.04 ) is done
__Learning Rate ( 0.09 ) is intialized
__Learning Rate ( 0.09 ) is done
__Learning Rate ( 0.16 ) is intialized
__Learning Rate ( 0.16 ) is done
__Learning Rate ( 0.25 ) is intialized
__Learning Rate ( 0.25 ) is done
__Learning Rate ( 0.36 ) is intialized
__Learning Rate ( 0.36 ) is done
__Learning Rate ( 0.49 ) is intialized
__Learning Rate ( 0.49 ) is done
__Learning Rate ( 0.64 ) is intialized
__Learning Rate ( 0.64 ) is done
__Learning Rate ( 0.81 ) is intialized
__Learning Rate ( 0.81 ) is done
__Learning Rate ( 1.0 ) is intialized
__Learning Rate ( 1.0 ) is done
__Learning Rate ( 1.21 ) is intialized
__Learning Rate ( 1.21 ) is done
__Learning Rate ( 1.44 ) is intialized
__Learning Rate ( 1.44 ) is done
__Learning Rate ( 1.69 ) is intialized
__Learning Rate ( 1.69 ) is done
__Learning Rate ( 1.96 ) is intialized
__Learning Rate ( 1.96 ) is done
__Learning Rate ( 2.25 ) is intia